##

## ANEXO - Traductor automático

A lo largo de este ejercicio se hacer uso de un modelo LLM entrenado principalmente con texto en inglés, por tanto, se debe interactuar con él en el mismo idioma.

Os dejo a continuación una pequeña ayuda para que podáis utilizar un traductor automático en caso de que se quiera interactuar con el modelo en castellano

In [1]:
!pip install googletrans==3.1.0a0

import googletrans
from googletrans import Translator
import pandas as pd

translator = Translator()

def traducir(text):
  return translator.translate(text, dest='en', src='es').text

def translate(text):
  return translator.translate(text, dest='es', src='en').text

Lo único que deberéis hacer es utilizar las funciones *traducir('texto')* y *translate('text')* para pasar del castellano al inglés y del inglés al castellano respectivamente.

# Nuestro primer LLM: Resumen de Diálogos

## 1 - Configurando el entorno

Este ejercicio hace uso de redes relativamente pesadas por lo que recomiendo que os aseguréis de que colab está usando un kernel apropiado (GPU o TPU)

A continuación, vampos a proceder a instalar los paquetes y datasets necesarios.

La siguiente celda puede tardar varios minutos...

In [2]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==2.6.0 \
    torchdata==0.11.0 --quiet

%pip install \
    transformers==4.41.0 \
    datasets==2.10.0  --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install \
    transformers==4.41.0 \
    datasets==2.10.0  --quiet

Note: you may need to restart the kernel to use updated packages.


Actualizamos datasets

In [4]:
%pip install --upgrade datasets --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Cargamos los datasets, el modelo LLM, el tokenizador y el configurador. No te preocupes si aún no entiendes estos componentes, los vamos a desgranar cuando los utilicemos

In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

/opt/anaconda3/envs/iagen/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<a name='2'></a>

## 2 - Resumen de diálogos sin Prompt Engineering

Para este ejercicio se generar resúmenes de diálogos con el modelo LLM pre-entrenado FLAN-T5 (https://www.datacamp.com/tutorial/flan-t5-tutorial) que descargaremos de Hugging Face (https://huggingface.co/docs/transformers/model_doc/flan-t5). La lista de modelos transformer disponibles puede encontrarse [aquí](https://huggingface.co/docs/transformers/index).

se utilizar algunos diálogos sencillos de la base de datos [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) También disponible en Hugging Face. Este dataset contiene más de 10,000 diálogos con sus etiquetas correspondientes (resúmenes y temática).

In [4]:
import os
import shutil
from datasets import load_dataset

dataset = load_dataset("knkarthick/dialogsum", download_mode="force_redownload")


Generating train split:   0%|          | 0/24920 [00:00<?, ? examples/s]/opt/anaconda3/envs/iagen/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:751: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
/opt/anaconda3/envs/iagen/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:751: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read_csv(xopen(filepath_or_buffer, "rb", download_config=download_config), **kwargs)
Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]/opt/anaconda3/envs/iagen/lib/python3.9/site-packages/datasets/download/streaming_download_manager.py:751: FutureWarning: The 'verbose' keyword in pd.read_csv is deprecated and will be removed in a future version.
  return pd.read

In [32]:

dataset['train'][0]

{'id': 'train_0',
 'dialogue': "#Person1#: Hi, Mr. Smith. I'm Doctor Hawkins. Why are you here today?\n#Person2#: I found it would be a good idea to get a check-up.\n#Person1#: Yes, well, you haven't had one for 5 years. You should have one every year.\n#Person2#: I know. I figure as long as there is nothing wrong, why go see the doctor?\n#Person1#: Well, the best way to avoid serious illnesses is to find out about them early. So try to come at least once a year for your own good.\n#Person2#: Ok.\n#Person1#: Let me see here. Your eyes and ears look fine. Take a deep breath, please. Do you smoke, Mr. Smith?\n#Person2#: Yes.\n#Person1#: Smoking is the leading cause of lung cancer and heart disease, you know. You really should quit.\n#Person2#: I've tried hundreds of times, but I just can't seem to kick the habit.\n#Person1#: Well, we have classes and some medications that might help. I'll give you more information before you leave.\n#Person2#: Ok, thanks doctor.",
 'summary': "Mr. Smith'

Como siempre, se trabajar un poquitín con los datos, aquí se dejar un poquitín de código para ver algunos ejemplos.

In [5]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

se cargar nuestro modelo, el FLAN - T5, para ello se crear una instancia de AutoModelForSeq2SeqLM con el método .from_pretrained()

In [6]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/opt/anaconda3/envs/iagen/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Para realizar la codificación y decodificación, necesitamos tokenizar el texto. Es decir, dividiremos los textos en unidades más pequeñas que puedan ser procesadas por modelos LLM.

A continuación se descargar el tokenizador para el modelo FLAN-T5 usando el método AutoTokenizer.from_pretrained(). El parámetro use_fast activa el tokenizador rápido. No es necesario entrar en detalles al respecto, pero si tenéis curiosidad se puede echar un ojo a la documentación.

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

se probar la codificación y decodificación de las frases tokenizadas:

In [8]:
sentence = "What time is it, Tom?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


se explorar qué tal funciona este LLM para resumir un diálogo sin hacer uso de Prompt Engineering. Recordad que Prompt Engineering es modificar el prompt (entrada de los datos) intentando mejorar la respuesta del modelo para una tarea determinada.

In [9]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

Como puedes ver las salidas del modelo tienen cierto sentido, pero no parece estar seguro de qué tarea se supone que debe realizar. Parece que simplemente constituye la siguiente oración del diálogo. Metemos un poquito de Prompt Engineering a ver qué pasa

## 3 - Resumen de Diálogos con Instruction Prompt

Prompt engineering es una herramienta muy importante en el uso de modelos básicos para la generación de texto. Os dejo un [link](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) si queréis refrescar conocimientos

<a name='3.1'></a>
### 3.1 - Inferencia Zero Shot con Instruction Prompt

Para pedirle a un modelo que realice una tarea - resumir un diálogo en este caso - necesitas coger dicho diálogo y convertirlo en un Instruction Prompt. A esto lo conocemos como **zero shot inference** ya que no utilizamos ningún ejemplo y le pedimos la tarea de forma directa.

se por ello (si tienes dificultad para entender el código, repasa f-string formatting en Python):

In [10]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Provide a one-line summary of the following conversation.

{dialogue}

Summary: ?
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Provide a one-line summary of the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary: ?
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:

Bastante mejor, no? Sin embargo, el modelo todavía no hace un resumen que realmente sintetice la información principal... se ver si podemos mejorar esto, os toca!

**Ejercicio:**

- Experimenta con el texto de `prompt` para ver cómo cambia la inferencia que podemos obtener del modelo. Cambia mucho el resultado cuando terminamos el diálogo sin ninguna especificación vs terminarlo con `Summary: ` Analiza los resultados y detalla si has realizado alguna otra prueba cuyo resultado te haya llamado la atención.
- Intenta cambiar el principio del `prompt`, reemplaza `Summarize the following conversation.` por algo diferente y comprueba cómo cambia la inferencia del sistema.

In [11]:
# EJERCICIO 1: EXPERIMENTACIÓN CON PROMPTS
print("="*80)
print("EJERCICIO 1: EXPERIMENTACIÓN CON PROMPTS")
print("="*80)

# Experimento 1: Sin especificación vs con "Summary:"
print("\n1. COMPARACIÓN: Sin especificación vs 'Summary:'")
print(dash_line)

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # Prompt sin especificación final
    prompt_sin_spec = f"""
Provide a one-line summary of the following conversation.

{dialogue}
    """

    # Prompt con "Summary:" al final
    prompt_con_summary = f"""
Provide a one-line summary of the following conversation.

{dialogue}

Summary:
    """

    print(f"\nEJEMPLO {i + 1}:")
    print(f"RESUMEN HUMANO: {summary}")
    
    # Genero respuestas
    inputs1 = tokenizer(prompt_sin_spec, return_tensors='pt')
    output1 = tokenizer.decode(
        model.generate(inputs1["input_ids"], max_new_tokens=50)[0],
        skip_special_tokens=True
    )
    
    inputs2 = tokenizer(prompt_con_summary, return_tensors='pt')
    output2 = tokenizer.decode(
        model.generate(inputs2["input_ids"], max_new_tokens=50)[0],
        skip_special_tokens=True
    )
    
    print(f"SIN ESPECIFICACIÓN: {output1}")
    print(f"CON 'Summary:': {output2}")
    print("-" * 50)



EJERCICIO 1: EXPERIMENTACIÓN CON PROMPTS

1. COMPARACIÓN: Sin especificación vs 'Summary:'
---------------------------------------------------------------------------------------------------

EJEMPLO 1:
RESUMEN HUMANO: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
SIN ESPECIFICACIÓN: Tom is late for the train.
CON 'Summary:': Tom is late for the train.
--------------------------------------------------

EJEMPLO 2:
RESUMEN HUMANO: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
SIN ESPECIFICACIÓN: #Person1: I'm thinking of upgrading my computer. #Person2: I'm not sure what exactly I would need. #Person1: I'd like to make up my own flyers and banners.
CON 'Summary:': #Person1#: I'm thinking of upgrading my computer.
--------------------------------------------------


In [12]:
# Experimento 2: Diferentes inicios de prompt
print("\n\n2. DIFERENTES INICIOS DE PROMPT")
print(dash_line)

prompt_variations = [
    "Summarize the following conversation.",
    "What is the main point of this dialogue?", 
    "Briefly explain what happened in this conversation.",
    "Create a short summary of this discussion.",
    "Tell me what this conversation is about in one sentence."
]

dialogue = dataset['test'][example_indices[0]]['dialogue']
summary = dataset['test'][example_indices[0]]['summary']

print(f"RESUMEN HUMANO: {summary}\n")

for j, variation in enumerate(prompt_variations):
    prompt = f"""
{variation}

{dialogue}

Summary:
    """
    
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], max_new_tokens=50)[0],
        skip_special_tokens=True
    )
    
    print(f"VARIACIÓN {j+1} - '{variation}':")
    print(f"RESULTADO: {output}\n")



2. DIFERENTES INICIOS DE PROMPT
---------------------------------------------------------------------------------------------------
RESUMEN HUMANO: #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

VARIACIÓN 1 - 'Summarize the following conversation.':
RESULTADO: The train is about to leave.

VARIACIÓN 2 - 'What is the main point of this dialogue?':
RESULTADO: Tom is late for the train. He has to catch the nine-thirty train.

VARIACIÓN 3 - 'Briefly explain what happened in this conversation.':
RESULTADO: Tom is late. He has to catch the nine-thirty train.

VARIACIÓN 4 - 'Create a short summary of this discussion.':
RESULTADO: Tom is late for work.

VARIACIÓN 5 - 'Tell me what this conversation is about in one sentence.':
RESULTADO: The train is about to leave.



In [13]:
"""
    - TODO 1: ¿Cambia mucho el resultado sin especificación vs terminándolo con Summary:?
        - No cambia significativamente. 
            - Ejemplo 1: ambos producen "Tom is late for the train" (idénticos). 
            - Ejemplo 2: sin especificación genera diálogo extendido, con "Summary:" produce 
            solo "#Person1#: I'm thinking of upgrading my computer" (más conciso pero ambos incorrectos).

    - TODO 2: Cambiar el principio del prompt - ¿cómo cambia la inferencia?
        - Diferentes inicios producen variaciones notables: 
            - "Summarize"                   → "The train is about to leave" (urgencia), 
            - "What is the main point?"     → respuesta más elaborada con detalles, 
            - "Briefly explain"             → versión condensada, 
            - "Create summary"              → añade información inexistente ("work"). 
            - Las preguntas directas generan respuestas más detalladas que las instrucciones imperativas.

    - Observación destacada: Todos los prompts mantienen el error sistemático de confundir personajes (Tom vs #Person1#), 
    sugiriendo limitaciones fundamentales del modelo que el prompt engineering superficial no puede resolver.

"""

'\n\nANÁLISIS EJERCICIO 1: EXPERIMENTACIÓN CON PROMPTS\n\n    - Hallazgo principal: Añadir "Summary:" no mejora la calidad del resumen - ambas variantes producen resultados \n    idénticos con el mismo error sistemático de confundir personajes (Tom vs #Person1#).\n\n    - Impacto de diferentes inicios: Las preguntas directas ("What is the main point?") generan respuestas más elaboradas \n    que las instrucciones imperativas, pero todos los prompts fallan en mantener roles correctos del diálogo.\n\n    - Implicación técnica: FLAN-T5-base presenta limitaciones en zero-shot para comprensión de diálogos multi-personaje, \n    sugiriendo que el problema es de capacidad del modelo más que de ingeniería de prompt superficial.\n\n    - Insight clave: La consistencia en los errores independientemente del formato indica que necesitamos estrategias más \n    sofisticadas como few-shot learning para guiar al modelo hacia una mejor comprensión contextual.\n\n'

<a name='3.2'></a>
### 3.2 - Inferencia Zero Shot usando los templates de Prompt de FLAN-T5

se usar un prompt ligeramente diferente. FLAN-T5 tiene muchos templates públicos para ciertas tareas [link](https://github.com/google-research/FLAN/tree/main/flan/v2). A continuación se usar uno de estos [prompts pre-built para FLAN-T5](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py):

In [14]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    prompt = f"""
Dialogue:

{dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
Tom is late for the train.

--------------

Como podemos ver este prompt obtiene mejores resultados pero le sigue costando extraer la esencia de las conversaciones... se ver si podemos mejorarlo!

<a name='4'></a>
## 4 - Resumen de Diálogos con One Shot y Few Shot

**One shot y few shot inference** consiste en darle al LLM uno o más ejemplos completos de pares prompt-response que se ajustan a tu tarea. A esto se le llama "in-context learning" y tiene como objetivo poner al modelo en un estado que facilite la resolución de tu tarea sin necesidad de ajustar pesos. se puede darle un pequeño repaso [en este blog de HuggingFace](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

<a name='4.1'></a>
### 4.1 - One Shot Inference

se crear una función que genere un prompt con ejemplos completos a partir de una lista de `example_indices_full` y añade al final el prompt que queremos que el modelo complete, `example_index_to_summarize`. se utilizar el mismo modelo FLAN-T5 de la sección [3.2](#3.2).

In [15]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Dialogue:

{dialogue}

What was going on?
{summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f"""
Dialogue:

{dialogue}

What was going on?
"""

    return prompt

Realizamos el prompt para hacer inferencias one shot:

In [16]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also ne

Ahora le pasamos este prompt para realizar la inferencia one shot:

In [17]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


### 4.2 - Few Shot Inference

se explorar few shot inference, es decir, la utilización de más de un ejemplo. Comenzaremos añadiendo dos ejemplos completos al prompt.

In [24]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.



Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. 

Ahora podemos pasarle este prompt al modelo para ver los resultados con few shot inference:

In [23]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Jack tells #Person1# that business communication is his favorite last year and #Person1# will check it.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


En este caso podemos ver que few shot no ha conseguido resultados mucho mejores que los que vimos con one shot. Además (y se puede probarlo) añadir más ejemplos no mejora demasiado en este ejemplo y rara vez se utilizan más de 5 o 6 ejemplos en ningún caso ya que no suele mejorar el resultado. Además, tenemos que estar seguros de que no se superar el contexto máximo de nuestro modelo, en este caso, 512 tokens.

Sin embargo, hemos podido ver que añadir uno ejemplo complto (one shot) si ha conseguido un modelo con más información y hemos mejorado de forma cualitativa el resumen.

**Ejercicio:**

Ahora te toca experimentar con few shot inferencing.
- Elige diferentes diálogos - Cambia los índices en la lista `example_indices_full` y el valor `example_index_to_summarize`. Analiza los resultados.
- Cambia el número de ejemplos (shots). Asegúrate de no sobrepasar el contexto máximo del modelo (512).

Qué puedes observar Cómo de bien funciona few shot inferencing para otros ejemplos

In [34]:
# EJERCICIO 2: FEW SHOT INFERENCING
print("="*80)
print("EJERCICIO 2: EXPERIMENTACIÓN CON FEW SHOT")
print("="*80)

# CAMBIO DE ÍNDICES EN example_indices_full y example_index_to_summarize
print("1. DIFERENTES COMBINACIONES DE DIÁLOGOS")
print(dash_line)

# Configuraciones a probar
test_configs = [
    {"indices": [40], "target": 200},
    {"indices": [80], "target": 200}, 
    {"indices": [120], "target": 200},
    {"indices": [40], "target": 180},
    {"indices": [80], "target": 180},
]

for i, config in enumerate(test_configs):
    example_indices_full = config["indices"]
    example_index_to_summarize = config["target"]
    
    prompt = make_prompt(example_indices_full, example_index_to_summarize)
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], max_new_tokens=50)[0],
        skip_special_tokens=True
    )
    
    target_summary = dataset['test'][example_index_to_summarize]['summary']
    print(f"CONFIG {i+1} - Ejemplos: {example_indices_full}, Target: {example_index_to_summarize}")
    print(f"REAL: {target_summary}")
    print(f"GENERADO: {output}\n")

# CAMBIO DEL NÚMERO DE EJEMPLOS (SHOTS)
print(dash_line)
print("2. VARIACIÓN EN NÚMERO DE SHOTS")
print(dash_line)

shot_configs = [
    [40],           # 1-shot
    [40, 80],       # 2-shot  
    [40, 80, 120],  # 3-shot
    [40, 80, 120, 150] # 4-shot
]

target = 200
target_summary = dataset['test'][target]['summary']
print(f"TARGET: {target} - RESUMEN REAL: {target_summary}\n")

for j, indices in enumerate(shot_configs):
    # Verifico longitud del prompt antes de generar
    test_prompt = make_prompt(indices, target)
    token_count = len(tokenizer.encode(test_prompt))
    
    # Respeto el límite de 512 tokens: prompt + max_new_tokens(50) debe ser < 512
    if token_count < 462:  # 512 - 50 = 462 tokens máximo para el prompt
        inputs = tokenizer(test_prompt, return_tensors='pt')
        output = tokenizer.decode(
            model.generate(inputs["input_ids"], max_new_tokens=50)[0],
            skip_special_tokens=True
        )
        print(f"{len(indices)}-SHOT (tokens: {token_count}): {output}")
    else:
        total_tokens = token_count + 50
        print(f"{len(indices)}-SHOT: EXCEDE LÍMITE DE 512 TOKENS")
        print(f"   Prompt: {token_count} + Generación: 50 = {total_tokens} tokens total")

print(f"\n{dash_line}")


EJERCICIO 2: EXPERIMENTACIÓN CON FEW SHOT
1. DIFERENTES COMBINACIONES DE DIÁLOGOS
---------------------------------------------------------------------------------------------------
CONFIG 1 - Ejemplos: [40], Target: 200
REAL: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
GENERADO: #Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.

CONFIG 2 - Ejemplos: [80], Target: 200
REAL: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
GENERADO: #Person1 wants to upgrade his system and hardware.

CONFIG 3 - Ejemplos: [120], Target: 200
REAL: #Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
GENERADO: #Person1 wants to upgrade his computer. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.

CONFIG 4 - Ejemplos: [40], Target: 180
REAL: Jack tells #Person1

In [ ]:
"""
    - ¿Qué puedes observar al cambiar el número de ejemplos?
        - Solo 1-shot es viable (392 tokens), mientras que 2-shot (631), 3-shot (819) y 4-shot (999 tokens) exceden 
        el límite de 512. Esto demuestra que FLAN-T5-base tiene severas limitaciones para few-shot learning con 
        diálogos largos.

    - ¿Cómo de bien funciona few shot inferencing para otros ejemplos?
        - Los resultados muestran calidad inconsistente: Target 200 genera resúmenes detallados pero con 
        alucinaciones ("CD-ROM drive", "painting program"), mientras Target 180 produce resúmenes extremadamente 
        simplificados ("Jack's classes were not bad") que pierden información clave. El ejemplo usado ([40] vs [80] vs [120]) 
        no mejora significativamente la precisión.

    - Observación clave: Few-shot (1-shot) invierte roles sistemáticamente - convierte "Person1 teaches Person2" en "Person1 
    wants to upgrade", manteniendo errores conceptuales independientemente de la configuración utilizada.

"""

## 5 - Configuración de los parámetros de inferencia para la generación

Puedes cambiar la configuración de parámetros del metodo `generate()` para modificar la salida del LLM. De momento, el único parámetro que hemos fijado ha sido `max_new_tokens=50`, lo que define el máximo número de tokens a generar. La lista completa de parámetros disponible puede encontrarse aquí: [Hugging Face Generation documentation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig).

Una forma de organizar los parámetros de configuración es usar la clase `GenerationConfig`.

**Ejercicio:**

Cambia la configuración de parámetros para investigar su influencia en el funcionamiento del modelo LLM.

Si ponemos el parámetro `do_sample = True` activaremos varias formas de decodificación que van a influir en la selección del siguiente token a partir de la probabilidad de distribución. A continuación puedes ajustar las salidas cambiando parámetros como `temperature`, `top_k` y `top_p`.

En la celda inferior tienes algunas ideas que probar, además se recomienda ponerse creativo y juguetear un poco con otras opciones, incluso con algunas un poco extremas!

Qué pasa si ponemos max_new_tokens a 10
Qué conseguimos mediante el cambio de temperature cuando do_sample = True
Qué otras cosas observas

In [36]:

# ¿Qué pasa si ponemos max_new_tokens a 10?
print("RESPUESTA DIRECTA: max_new_tokens=10")
print("="*60)
generation_config_10 = GenerationConfig(
    max_new_tokens=10,
    decoder_start_token_id=tokenizer.pad_token_id
)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output_10 = tokenizer.decode(
    model.generate(inputs["input_ids"], generation_config=generation_config_10)[0],
    skip_special_tokens=True
)
print(f"max_new_tokens=10: {output_10}")
print(f"Comparado con baseline: {summary}")
print("OBSERVACIÓN: Mucho más conciso y elimina alucinaciones\n")

print("EXPERIMENTOS CON top_k y top_p:")
print("="*60)

# Experimento con top_k
print("TOP_K VARIATIONS:")
top_k_configs = [5, 10, 50]
for k in top_k_configs:
    generation_config = GenerationConfig(
        max_new_tokens=50,
        do_sample=True,
        temperature=1.0,
        top_k=k,
        decoder_start_token_id=tokenizer.pad_token_id
    )
    
    inputs = tokenizer(few_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], generation_config=generation_config)[0],
        skip_special_tokens=True
    )
    
    print(f"top_k={k}: {output}\n")

# Experimento con top_p
print("TOP_P VARIATIONS:")
top_p_configs = [0.8, 0.9, 0.95]
for p in top_p_configs:
    generation_config = GenerationConfig(
        max_new_tokens=50,
        do_sample=True,
        temperature=1.0,
        top_p=p,
        decoder_start_token_id=tokenizer.pad_token_id
    )
    
    inputs = tokenizer(few_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], generation_config=generation_config)[0],
        skip_special_tokens=True
    )
    
    print(f"top_p={p}: {output}\n")

# ¿Qué conseguimos mediante el cambio de temperature?
print("RESPUESTA DIRECTA: Comparación de temperaturas")
print("="*60)
temp_comparison = [0.1, 1.0, 2.0]
for temp in temp_comparison:
    generation_config = GenerationConfig(
        max_new_tokens=30,
        do_sample=True,
        temperature=temp,
        decoder_start_token_id=tokenizer.pad_token_id
    )
    
    inputs = tokenizer(few_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], generation_config=generation_config)[0],
        skip_special_tokens=True
    )
    
    print(f"temperature={temp}: {output}")
print("OBSERVACIÓN: A mayor temperature, mayor creatividad pero menor coherencia\n")

# CONFIGURACIONES EXTREMAS
print("CONFIGURACIONES EXTREMAS:")
print("="*60)

extreme_configs = [
    {
        "name": "Ultra restrictivo (top_k=1, temp=0.01)",
        "config": GenerationConfig(
            max_new_tokens=50,
            do_sample=True,
            temperature=0.01,
            top_k=1,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    },
    {
        "name": "Ultra creativo (temp=2.0, top_p=0.99)",
        "config": GenerationConfig(
            max_new_tokens=50,
            do_sample=True,
            temperature=2.0,
            top_p=0.99,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    },
    {
        "name": "Muy corto + caótico (5 tokens, temp=3.0)",
        "config": GenerationConfig(
            max_new_tokens=5,
            do_sample=True,
            temperature=3.0,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    },
    {
        "name": "Híbrido restrictivo (top_k=3, top_p=0.5)",
        "config": GenerationConfig(
            max_new_tokens=50,
            do_sample=True,
            temperature=1.0,
            top_k=3,
            top_p=0.5,
            decoder_start_token_id=tokenizer.pad_token_id
        )
    }
]

for experiment in extreme_configs:
    inputs = tokenizer(few_shot_prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(inputs["input_ids"], generation_config=experiment["config"])[0],
        skip_special_tokens=True
    )
    
    print(f"{experiment['name']}:")
    print(f"RESULTADO: {output}\n")

print("="*60)


RESPUESTA DIRECTA: max_new_tokens=10
max_new_tokens=10: #Person1 wants to upgrade his system.
Comparado con baseline: Jack tells #Person1# that business communication is his favorite last year and #Person1# will check it.
OBSERVACIÓN: Mucho más conciso y elimina alucinaciones

EXPERIMENTOS CON top_k y top_p:
TOP_K VARIATIONS:
top_k=5: People are going to upgrade their computers and make their software more powerful and faster.ones want to upgrade their systems as well.ones want to add a painting program to their software.ones wants to add a CD

top_k=10: #Person1 recommends upgrading the software to make it more powerful.nsia has a CD-ROM drive and a faster processor. #Person1 wants to use a software on Cds.n

top_k=50: Person1 has mentioned adding a painting program to his computer software.a also considers his need to upgrade his hardware.a offers a CD-ROM drive for upgrading the PC and system.a is prepared to help him select

TOP_P VARIATIONS:
top_p=0.8: The new software is coming o

In [ ]:
"""
    - ¿Qué pasa si ponemos max_new_tokens a 10?
        - Resultado: "#Person1 wants to upgrade his system" - Produce el resumen más limpio y preciso, eliminando 
        completamente las alucinaciones sobre CD-ROM y painting programs que aparecen en versiones más largas.

    - ¿Qué conseguimos mediante el cambio de temperature cuando do_sample = True?
        - Temperature 0.1: Texto estructurado pero repetitivo. Temperature 1.0: Introduce errores 
        lógicos ("turn in computer software"). Temperature 2.0: Genera texto casi incoherente sobre "SD cards" y 
        aplicaciones inexistentes.

    - ¿Qué otras cosas observas?
        - Los parámetros top_k y top_p producen degradación progresiva - fragmentos como ".te is unsure", ".raeces.com", 
        ".snel1" indican que el modelo genera tokens corruptos. Las configuraciones extremas revelan colapso total: "temp=3.0" 
        produce solo "#Person#:" y configuraciones ultra restrictivas generan loops infinitos (".ta is not available.ta is 
        not a CD").
"""